## Vistual inertia scheduling

vittual inertia scheduling (vis) is inherited from dcopf in opf.py

base class: dcopf <br>
vis1: dcopf + Pvsg <br>
vis2: dcopf + RoCof and fnadir + Pvsg <br>


In [315]:
import andes
import os

from statistics import fmean
from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import gurobipy as gb
import pandas as pd
import numpy as np
import logging

logger = logging.getLogger(__name__)

from opf import dcopf

In [316]:
class vis1(dcopf):
    """
    vis0: fixed vsg up and down reserve
    """

    def __init__(self, name='dcopf', norm=None, nn=None, nn_num=64, dpe=1):
        """
        Parameter
        ---------
        name: str
            name 
        norm: dict
            normalization dict for function fnaidir and ppeak 
            {
                Mvsg: [mean, std],
                Dvsg: [mean, std],
                Fg: [mean, std],
                Rg: [mean, std],
                Minv: [mean, std],
                Dinv: [mean, std],
            }
        nn: neural
        nn_num: integer
            number of MLP nuerols (assume single layer MLP)
        """
        super().__init__(name)

        self.norm = norm # TO DO ...
        self.nn = nn # TO DO ...
        self.nn_num = nn_num

        self.fnadir = 0.01 # 0.6Hz
        self.rocof = 0.01 # 0.6Hz/s
        self.dpe = dpe # delta Pe


    def from_andes(self, ssa, typeII=None):
        """
        ssa: andes model
        typeII: idx of typeII generator, vsg inverter
            i.g. ['PV_6', 'PV_7']
        """      
        super().from_andes(ssa)

        # define typeII, defalt typeI: type=1
        self.gen['type'] = 1
        if typeII:
            for idx in typeII:
                row = self.gen[self.gen['idx'] == idx].index[0]
                self.gen['type'].iloc[row] = 2

        self.gen['p_pre'] = 0
        self.gen['band'] = self.gen['pmax'] - self.gen['pmin']
        self.gen['Sn'] /= self.mva  # normalize Sn
        self.gen['fg'] = 1
        self.gen['K'] = 0
        # self.gen['M'] = 0
        # self.gen['D'] = 0
        # self.gen['R'] = 0.05
        # self.gen['Mvsg'] = 0
        # self.gen['Dvsg'] = 0

        genrow = ssa.GENROU.as_df()
        regc = ssa.REGCV2.as_df()
        tgov = ssa.TGOV1N.as_df()
        tgov.rename(columns={'idx':'gov', 'syn':'idx'}, inplace=True)
        regc.rename(columns={'idx':'vsg', 'gen':'idx', 'M':'Mvsg', 'D': 'Dvsg'},  inplace=True)

        # merge tgov R to genrow based on idex in tgover and syn in tgov
        genrow = pd.merge(left=genrow, right=tgov[['idx', 'R']], on='idx', how='left')

        # merge M, D, R in genrow to self.gen based on gen
        genrow.rename(columns={'idx': 'syn', 'gen': 'idx'}, inplace=True)
        self.gen = pd.merge(left=self.gen, right=genrow[['idx', 'M','D', 'R']], on='idx', how='left')

        # merge Mvsg, Dvsg, in genrow to self.gen based on gen
        self.gen = pd.merge(left=self.gen, right=regc[['idx', 'Mvsg', 'Dvsg']], on='idx', how='left')

        self.gen.fillna(0, inplace=True)

        # update dict after revising pandas dataframe
        self.update_dict()

        # TODO: buidl global index for typeI/II gen


    def build(self):
        # self.data_check()

        # --- build gurobi model ---
        self.update_dict()
        self.mdl = gb.Model(self.name)
        self.mdl = self._build_vars(self.mdl)
        self.mdl = self._build_obj(self.mdl)
        self.mdl = self._build_cons(self.mdl)
        logger.info('Successfully build vis0 model.')



    def _build_vars(self, mdl):
        GEN = self.gendict.keys()

        # --- uncontrollable generators limit to p0 ---
        gencp = self.gen.copy()
        gencp['pmax'][gencp.ctrl == 0] = gencp['p0'][gencp.ctrl == 0]
        gencp['pmin'][gencp.ctrl == 0] = gencp['p0'][gencp.ctrl == 0]
        # --- offline geenrators limit to 0 ---
        gencp['pmax'][gencp.u == 0] = 0
        gencp['pmin'][gencp.u == 0] = 0

        # --- gen: pg ---
        self.pg = mdl.addVars(GEN, name='pg', vtype=gb.GRB.CONTINUOUS, obj=0,
                              ub=gencp.pmax.tolist(), lb=gencp.pmin.tolist())
        # --- RegUp, RegDn --- !!! modify inverter reserve up and down in andes file, prd=0
        self.pru = mdl.addVars(GEN, name='pru', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=gencp.band.tolist(), lb=[0] * gencp.shape[0])
        self.prd = mdl.addVars(GEN, name='prd', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=gencp.band.tolist(), lb=[0] * gencp.shape[0])
        
        # --- Mvsg, Dvsg ---
        gendict = self.gendict
        gendict_II = dict()
        for (new_key, new_value) in gendict.items():
            if new_value['type'] == 2:
                gendict_II[new_key] = new_value
        vsg = gendict_II.keys()

        self.Mvsg = mdl.addVars(vsg, name='Mvsg', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=[4]*len(vsg), lb=[0]*len(vsg))
        self.Dvsg = mdl.addVars(vsg, name='Dvsg', vtype=gb.GRB.CONTINUOUS, obj=0,
                               ub=[5]*len(vsg), lb=[0]*len(vsg))

        # --- a and z for ml assisted linearization ---
        # 'ap, zp' for vsg power; 'af, zf' for frequency nadir
        ap = []
        zp = []
        for i in range(self.nn_num):
            ap.append('ap'+str(i))
            zp.append('zp'+str(i))
        self.ap = mdl.addVars(ap, name='ap', vtype=gb.GRB.BINARY)
        self.zp = mdl.addVars(zp, name='zp', vtype=gb.GRB.CONTINUOUS)
        print('Successfully build var.')

        return mdl


    def _build_obj(self, mdl):
        GEN = self.gendict.keys()
        gendict = self.gendict
        costdict = self.costdict

        # --- minimize generation cost ---
        cost_pg = sum(self.pg[gen] * costdict[gen]['c1']
                      + self.pg[gen] * self.pg[gen] * costdict[gen]['c2']
                      + costdict[gen]['c0'] * gendict[gen]['u']  # online status
                      for gen in GEN)

        # --- RegUp, RegDn cost ---
        cost_ru = sum(self.pru[gen] * costdict[gen]['cru'] for gen in GEN)
        cost_rd = sum(self.prd[gen] * costdict[gen]['crd'] for gen in GEN)
        cost_vsg = cost_ru + cost_rd

        self.obj = mdl.setObjective(expr=cost_pg + cost_vsg, sense=gb.GRB.MINIMIZE)
        print('Successfully build obj.')
        return mdl


    def _build_cons(self, mdl):
        # --- var idx ---
        ptotal = self.load.p0.sum()

        gendict = self.gendict
        linedict = self.linedict
        gen_gsfdict = self.gen_gsfdict

        GEN = gendict.keys()
        LINE = linedict.keys()

        # --- filter Type II gen ---
        gendict_I, gendict_II= dict(), dict()
        for (new_key, new_value) in gendict.items():
            if new_value['type'] == 1:
                gendict_I[new_key] = new_value
        for (new_key, new_value) in gendict.items():
            if new_value['type'] == 2:
                gendict_II[new_key] = new_value
        GENI = gendict_I.keys()
        GENII = gendict_II.keys()

        # --- Synthetic M/D/F/R ---
        Msys = sum(gendict[gen]['Sn'] * gendict[gen]['M'] for gen in GENI)
        Msys += sum(gendict[gen]['Sn'] * self.Mvsg[gen] for gen in GENII)
        Msys /= sum(gendict[gen]['Sn'] for gen in gendict.keys())

        Dsys = sum(gendict[gen]['Sn'] * gendict[gen]['D'] for gen in GENI)
        Dsys += sum(gendict[gen]['Sn'] * self.Dvsg[gen] for gen in GENII)
        Dsys /= sum(gendict[gen]['Sn'] for gen in gendict.keys())

        Rsys = sum(gendict[gen]['K'] / gendict[gen]['R'] * gendict[gen]['Sn'] for gen in GENI)
        Rsys /= sum(gendict[gen]['Sn'] for gen in GENI)

        Fsys = sum(gendict[gen]['K'] / gendict[gen]['R'] * self.pg[gen] for gen in GENI)
        Fsys /= sum(gendict[gen]['Sn'] for gen in GENI)

        # --- RoCof ----

        # --- fnadir ----

        # --- VSG ppeak ---

        # --- 01 power balance ---
        p_sum = sum(self.pg[gen] for gen in GEN)
        mdl.addConstr(p_sum == ptotal, name='PowerBalance')

        # --- 02 line limits ---
        for line in LINE:
            lhs1 = sum(self.pg[gen] * gen_gsfdict[gen][line] for gen in GEN)
            mdl.addConstr(lhs1+linedict[line]['sup'] <= linedict[line]['rate_a'], name=f'{line}_U')
            mdl.addConstr(lhs1+linedict[line]['sup'] >= -linedict[line]['rate_a'], name=f'{line}_D')


        print('Successfully build cons.')
        return mdl


    def get_res(self):
        """
        Get resutlts, can be used after mdl.optimize().

        Returns
        -------
        DataFrame
            The output DataFrame contains setpoints ``pg``

        """
        self.build()
        self.mdl.optimize()
        # --- check if mdl is sovled ---
        if not hasattr(self.pg[self.gen.idx[0]], 'X'):
            logger.warning('RTED has no valid resutls!')
            pg = [0] * self.gen.shape[0]
            pru = [0] * self.gen.shape[0]
            prd = [0] * self.gen.shape[0]
        else:
            logger.warning('Successfully solve RTED.')
            # --- gather data --
            pg = []
            pru = []
            prd = []
            for gen in self.gendict.keys():
                pg.append(self.pg[gen].X)
                pru.append(self.pru[gen].X)
                prd.append(self.prd[gen].X)
            # --- cost ---
            self.res_cost = self.mdl.getObjective().getValue()
            logger.info(f'Total cost={np.round(self.res_cost, 3)}')
        # --- build output table ---
        dcres = pd.DataFrame()
        dcres['gen'] = self.gen['idx']
        dcres['pg'] = pg
        dcres['pru'] = pru
        dcres['prd'] = prd
        dcres.fillna(0, inplace=True)
        return dcres




## Main (Test)

In [317]:
# get andes case from excel
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee14_vis.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)

REGCV2: unused data {'KpId': 50, 'KiId': 100, 'KpIq': 50, 'KiIq': 100}
REGCV2: unused data {'KpId': 50, 'KiId': 100, 'KpIq': 50, 'KiIq': 100}


In [318]:
ss = vis1()

In [319]:
ss.from_andes(ssa, ['PV_6', 'PV_7'])

In [320]:
ss.gen

,idx,u,name,Sn,Vn,bus,p0,pmax,pmin,v0,...,type,p_pre,band,fg,K,M,D,R,Mvsg,Dvsg
0,PV_2,1.0,PV_2,1.0,69.0,2,0.40000,0.5,0.1,1.03,...,1,0,0.4,1,0,8.0,0.0,0.12,0.0,0.0
1,PV_3,1.0,PV_3,1.0,69.0,3,0.40000,0.5,0.1,1.01,...,1,0,0.4,1,0,5.0,0.0,0.12,0.0,0.0
2,PV_4,1.0,PV_4,1.0,138.0,6,0.30000,1.0,0.1,1.03,...,1,0,0.9,1,0,5.0,0.0,0.05,0.0,0.0
3,PV_5,1.0,PV_5,1.0,69.0,8,0.30000,0.5,0.1,1.03,...,1,0,0.4,1,0,10.0,0.0,0.05,0.0,0.0
4,PV_6,1.0,PV_6,1.0,138.0,14,0.10000,0.1,0.0,1.01,...,2,0,0.1,1,0,0.0,0.0,0.00,0.0,3.0
5,PV_7,1.0,PV_7,1.0,138.0,12,0.10000,0.1,0.0,1.01,...,2,0,0.1,1,0,0.0,0.0,0.00,6.0,2.0
6,Slack_1,1.0,Slack_1,1.0,69.0,1,0.81442,3.0,0.5,1.03,...,1,0,2.5,1,0,8.0,0.0,0.12,0.0,0.0


In [321]:
# ss.gendict

In [322]:
ss.norm

In [323]:
a_test = []
for i in range(64):
    a_test.append('a'+str(i))
# a_test

In [324]:
# ss.Mvsg

In [325]:
# ss.pru

In [326]:
ss.costdict

{'PV_2': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_3': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_4': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_5': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_6': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_7': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'Slack_1': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0}}

In [327]:
ss.gendict.keys()

dict_keys(['PV_2', 'PV_3', 'PV_4', 'PV_5', 'PV_6', 'PV_7', 'Slack_1'])

In [328]:
ss.build()

# ss.mdl.optimize()

Successfully build var.
Successfully build obj.
Successfully build cons.


In [329]:
ss.mdl.display()

Minimize
  <gurobi.LinExpr: 0.0>
Subject To


In [330]:
ss.get_res()

Successfully build var.
Successfully build obj.
Successfully build cons.
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 33 rows, 153 columns and 167 nonzeros
Model fingerprint: 0xf292f6d9
Variable types: 89 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [4e-04, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 5e+00]
  RHS range        [1e-01, 4e+00]
Found heuristic solution: objective 2.2870000
Presolve removed 33 rows and 153 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 2.287 2.287 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.287000000000e+00, best bound 2.287000000000e+00, gap 0.0000%


Successfully solve RTED.


,gen,pg,pru,prd
0,PV_2,0.100,0.0,0.0
1,PV_3,0.100,0.0,0.0
2,PV_4,0.100,0.0,0.0
3,PV_5,0.100,0.0,0.0
4,PV_6,0.000,0.0,0.0
5,PV_7,0.000,0.0,0.0
6,Slack_1,1.887,0.0,0.0


In [331]:
ss.load.p0.sum()

2.287